# Constantes

In [ ]:
# |export

AUDIO_PATH = "audios"

# list m4a files

In [ ]:
# |export

from download import get_audio_path
import os, glob


def list_m4a_files(audio_path=AUDIO_PATH):
    fullpath = get_audio_path(audio_path)

    return glob.glob(os.path.join(fullpath, "*.m4a"))

In [ ]:
list_m4a_files()[:5]

['/home/guillaume/git/lmelp/audios/14007-05.01.2025-ITEMA_23985183-2025F4007S0005-27.m4a']

# extract whisper

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


def extract_whisper(mp3_filename):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

    dataset = load_dataset(
        "distil-whisper/librispeech_long", "clean", split="validation"
    )
    sample = dataset[0]["audio"]

    result = pipe(
        mp3_filename,
        return_timestamps=True,
    )

    return result["text"]

In [ ]:
m4a_1 = list_m4a_files()[0]

whisper = extract_whisper(m4a_1)

whisper[:150]

Device set to use cpu
/usr/share/libdrm/amdgpu.ids: No such file or directory
amdgpu: unknown (family_id, chip_external_rev): (148, 10)
libGL error: failed to create dri screen
libGL error: failed to load driver: radeonsi
/home/guillaume/miniforge3/envs/whisper/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_ke

' France Inter Le masque et la plume Excellente année à toutes et tous, auditeurs, auditrices et tous les autres, pour reprendre un mot de David Bowie.'